---

# **CRIPTOGRAFÍA Y SEGURIDAD DE LA INFORMACIÓN**

---

## **UNIVERSIDAD CENTRAL DEL ECUADOR**

**FACULTAD DE INGENIERÍA Y CIENCIAS APLICADAS**
**CARRERA DE COMPUTACIÓN**

---

## **TRABAJO PRÁCTICO – PKI (Public Key Infrastructure)**

**Implementación práctica de infraestructura de clave pública (PKI) utilizando RSA, certificados X.509 y firma digital**

---

### **Integrantes:**

*   **Kelly Ledesma**
*   **Stalin Acurio**
*   **Angelo Silva**
*   **Edison Enríquez**

### **Docente:**

**Ing. Giovanny Moncayo**

---


---

## **Escenario PKI: Firma digital de documentos entre Marta y Carlos**
### 1. Descripción general del escenario

En este ejemplo se modela un caso típico de uso de una *Public Key Infrastructure (PKI)* dentro de una organización: la **firma digital de documentos** por parte de un emisor (Marta) y la **verificación de la firma** por parte de un receptor.

El objetivo es garantizar que Carlos pueda comprobar que:

1. El documento fue realmente emitido por Marta (*autenticidad*).
2. El contenido no ha sido modificado desde que Marta lo firmó (*integridad*).
3. Marta no pueda negar posteriormente haber firmado ese documento (*no repudio*, en un entorno con políticas legales adecuadas).

Para ello se utilizan **claves criptográficas**, **certificados digitales X.509** y una **infraestructura de clave pública (PKI)**.

---

### 2. Roles involucrados

En este escenario intervienen los siguientes actores:

* **Marta**
  Emisora del documento. Posee un **par de claves asimétricas** (clave privada y clave pública) y un **certificado digital** que vincula su identidad con su clave pública.

* **Carlos**
  Receptor del documento. No necesita su propio par de claves para este ejemplo concreto; su función es **verificar la firma** de Marta usando el certificado público de Marta.

* **Autoridad Certificadora (CA)**
  En una PKI real, la CA es la entidad que **firma y emite certificados** para los usuarios.
  En este ejemplo de laboratorio, **simularemos este rol mediante un certificado autofirmado (self-signed)** emitido por Marta, para simplificar la implementación sin perder el enfoque didáctico.

---

### 3. Flujo de alto nivel del proceso

1. **Generación del par de claves de Marta**

   * Marta genera una clave privada y una clave pública (por ejemplo, RSA de 2048 bits).
   * La clave privada se mantiene en secreto y nunca se comparte.
   * La clave pública será incluida en su certificado.

2. **Emisión del certificado digital de Marta**

   * Se construye un certificado X.509 con los datos de identidad de Marta (nombre, organización, etc.) y su clave pública.
   * En una PKI real, este certificado sería firmado por la CA.
   * En este ejemplo, el certificado será **autofirmado** usando la propia clave privada de Marta (self-signed), lo cual es aceptable para fines académicos.

3. **Distribución del certificado de Marta a Carlos**

   * Carlos recibe el **certificado digital de Marta** por un canal confiable (por ejemplo, repositorio interno de la organización o un directorio de certificados).
   * Carlos **confía** en ese certificado para verificar las firmas de Marta dentro del contexto de este laboratorio.

4. **Firma digital del documento por parte de Marta**

   * Marta toma el contenido del documento (por ejemplo, un contrato en formato texto) y calcula un **hash** del mismo (por ejemplo, usando SHA-256).
   * Con su **clave privada**, genera la **firma digital** del hash utilizando un esquema seguro (por ejemplo, RSA-PSS).
   * Marta envía a Carlos:

     * El **documento en claro**
     * La **firma digital**
     * Su **certificado digital**

5. **Verificación de la firma por parte de Carlos**

   * Carlos extrae la **clave pública de Marta** del certificado digital recibido.
   * Calcula de nuevo el **hash** del documento.
   * Usa la clave pública para **verificar la firma**:

     * Si la verificación es correcta, Carlos concluye que:

       * El documento **no ha sido modificado** (*integridad*).
       * La firma fue generada con la clave privada asociada al certificado de Marta (*autenticidad*).
     * Si la verificación falla, Carlos asume que el documento fue alterado o la firma no es válida.

---

### 4. Cómo se reflejará esto en el código

Cuando reescribamos el código en el notebook, podemos mapear directamente este flujo:

1. **Generar par de claves de Marta**

   ```python
   marta_private_key, marta_public_key = generate_key_pair()
   ```

2. **Crear certificado (autofirmado) para Marta**

   ```python
   marta_cert = create_self_signed_certificate(marta_private_key, marta_public_key, common_name="Marta")
   ```

3. **Simular la “distribución” del certificado a Carlos**

   * En el código simplemente usaremos `marta_cert` desde el mismo programa, pero en el informe explicaremos que en un entorno real se distribuiría a Carlos.

4. **Marta firma un mensaje**

   ```python
   message = "Contrato de prestación de servicios..."
   signature = sign_message(marta_private_key, message)
   ```

5. **Carlos verifica la firma usando la clave pública del certificado de Marta**

   ```python
   marta_public_from_cert = marta_cert.public_key()
   verify_signature(marta_public_from_cert, message, signature)
   ```

---

In [3]:
# ============================================================
#   EJEMPLO COMPLETO DE PKI (Escenario: Marta → carlos)
#   Firma digital y verificación con RSA y certificados X.509
#   Incluye una Autoridad Certificadora (CA) independiente
# ============================================================

from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography import x509
from cryptography.x509.oid import NameOID
from datetime import datetime, timedelta, timezone


# ============================================================
# 1. Generación del par de claves de Marta
# ============================================================

def generate_rsa_key_pair():
    """
    Genera un par de claves RSA (privada + pública).
    Retorna: private_key, public_key
    """
    private_key = rsa.generate_private_key(
        public_exponent=65537,          # estándar recomendado
        key_size=2048                   # tamaño mínimo aceptable hoy
    )
    public_key = private_key.public_key()
    print("Par de claves RSA generado correctamente.")
    return private_key, public_key


# ============================================================
# 2. Funciones para Creación y Emisión de Certificados X.509
# ============================================================

def create_self_signed_certificate(private_key, public_key, common_name: str, days_valid: int = 365):
    """
    Crea un certificado X.509 autofirmado (self-signed).
    Útil para la CA raíz en un entorno de laboratorio.
    """
    subject = issuer = x509.Name([
        x509.NameAttribute(NameOID.COUNTRY_NAME, "EC"),
        x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, "Pichincha"),
        x509.NameAttribute(NameOID.LOCALITY_NAME, "Quito"),
        x509.NameAttribute(NameOID.ORGANIZATION_NAME, "Universidad Central del Ecuador"),
        x509.NameAttribute(NameOID.COMMON_NAME, common_name),
    ])

    not_before = datetime.now(timezone.utc)
    not_after = not_before + timedelta(days=days_valid)

    certificate = (
        x509.CertificateBuilder()
        .subject_name(subject)
        .issuer_name(issuer)
        .public_key(public_key)
        .serial_number(x509.random_serial_number())
        .not_valid_before(not_before)
        .not_valid_after(not_after)
        .add_extension(
            x509.SubjectAlternativeName([x509.DNSName("localhost")]),
            critical=False
        )
        .sign(private_key, hashes.SHA256(), padding.PKCS1v15()) # Explicitly use PKCS1v15 for cert signing
    )

    print(f"Certificado X.509 autofirmado para '{common_name}' creado correctamente.")
    return certificate

def issue_certificate(user_private_key, user_public_key, ca_private_key, ca_cert, common_name: str, days_valid: int = 365):
    """
    Emite un certificado X.509 para un usuario, firmado por una CA.
    """
    subject = x509.Name([
        x509.NameAttribute(NameOID.COUNTRY_NAME, "EC"),
        x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, "Pichincha"),
        x509.NameAttribute(NameOID.LOCALITY_NAME, "Quito"),
        x509.NameAttribute(NameOID.ORGANIZATION_NAME, "Universidad Central del Ecuador"),
        x509.NameAttribute(NameOID.COMMON_NAME, common_name),
    ])

    not_before = datetime.now(timezone.utc)
    not_after = not_before + timedelta(days=days_valid)

    certificate = (
        x509.CertificateBuilder()
        .subject_name(subject)
        .issuer_name(ca_cert.subject) # El emisor es la CA
        .public_key(user_public_key)
        .serial_number(x509.random_serial_number())
        .not_valid_before(not_before)
        .not_valid_after(not_after)
        .add_extension(
            x509.SubjectAlternativeName([x509.DNSName("localhost")]),
            critical=False
        )
        .sign(ca_private_key, hashes.SHA256(), padding.PKCS1v15()) # La CA firma con su clave privada
    )

    print(f"Certificado X.509 para '{common_name}' emitido por '{ca_cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value}' creado correctamente.")
    return certificate


def verify_certificate_chain(user_cert, ca_cert) -> bool:
    """
    Verifica si un certificado de usuario fue válidamente emitido por una CA.
    Retorna True si la verificación es exitosa, False en caso contrario.
    """
    try:
        # Verify the user certificate's signature using the CA's public key
        ca_public_key = ca_cert.public_key()
        ca_public_key.verify(
            user_cert.signature,
            user_cert.tbs_certificate_bytes,
            padding.PKCS1v15(),  # X.509 certificates generally use PKCS1v15 padding for signatures
            user_cert.signature_hash_algorithm
        )
        # Optionally, check common name or other attributes if necessary
        if user_cert.issuer != ca_cert.subject:
            raise ValueError("Issuer name in user certificate does not match CA subject name.")

        print("✔ Certificado del usuario verificado con la CA.")
        return True
    except Exception as e:
        print("✘ ERROR: El certificado del usuario NO es válido o no fue emitido por la CA esperada.")
        print("Detalles del error:", e)
        return False


# ============================================================
# 3. Generación de claves y certificado autofirmado de la CA
# ============================================================

ca_private_key, ca_public_key = generate_rsa_key_pair()
ca_cert = create_self_signed_certificate(
    ca_private_key,
    ca_public_key,
    common_name="Autoridad Certificadora Central"
)


# ============================================================
# 4. Generación de claves de Marta y emisión de su certificado por la CA
# ============================================================

marta_private_key, marta_public_key = generate_rsa_key_pair()

marta_cert = issue_certificate(
    marta_private_key,
    marta_public_key,
    ca_private_key,
    ca_cert,
    common_name="Marta"
)


# ============================================================
# 5. Firma digital del mensaje por parte de Marta
# ============================================================

def sign_message(private_key, message: str) -> bytes:
    """
    Firma un mensaje usando RSA-PSS + SHA-256.
    Retorna los bytes de la firma.
    """
    signature = private_key.sign(
        message.encode("utf-8"),
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    print("\nMensaje firmado por Marta.")
    return signature



# Ejemplo de mensaje a firmar
message = "Este es un documento importante para carlos."
signature = sign_message(marta_private_key, message)



# ============================================================
# 6. Verificación de firma por parte de carlos
# ============================================================

def verify_signature(public_key, message: str, signature: bytes) -> bool:
    """
    Verifica una firma digital usando la clave pública.
    Retorna True si la firma es válida, False caso contrario.
    """
    try:
        public_key.verify(
            signature,
            message.encode("utf-8"),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print("✔ FIRMA VERIFICADA: el mensaje es auténtico y no ha sido alterado.")
        return True
    except Exception as e:
        print("✘ ERROR: la firma NO es válida.")
        print("Detalles del error:", e)
        return False



# carlos recibe el certificado de la CA y el certificado de Marta.
# Primero, carlos verifica la validez del certificado de Marta usando el certificado de la CA.
print("\n=== carlos verifica el certificado de Marta con la CA ===")
marta_cert_verified_by_carlos = verify_certificate_chain(marta_cert, ca_cert)

# Si el certificado de Marta es válido, carlos extrae la clave pública de Marta y verifica la firma del mensaje.
if marta_cert_verified_by_carlos:
    carlos_public_key = marta_cert.public_key()

    print("\n=== carlos verifica la firma del mensaje original ===")
    verify_signature(carlos_public_key, message, signature)

    print("\n=== carlos verifica la firma con un mensaje alterado ===")
    verify_signature(carlos_public_key, message + " (alterado)", signature)
else:
    print("\nNo se puede verificar la firma del mensaje porque el certificado de Marta no es válido.")



# ============================================================
# 7. Resultado final
# ============================================================
print("\nProceso completo de PKI ejecutado correctamente (Marta → carlos con CA).")

Par de claves RSA generado correctamente.
Certificado X.509 autofirmado para 'Autoridad Certificadora Central' creado correctamente.
Par de claves RSA generado correctamente.
Certificado X.509 para 'Marta' emitido por 'Autoridad Certificadora Central' creado correctamente.

Mensaje firmado por Marta.

=== carlos verifica el certificado de Marta con la CA ===
✔ Certificado del usuario verificado con la CA.

=== carlos verifica la firma del mensaje original ===
✔ FIRMA VERIFICADA: el mensaje es auténtico y no ha sido alterado.

=== carlos verifica la firma con un mensaje alterado ===
✘ ERROR: la firma NO es válida.
Detalles del error: 

Proceso completo de PKI ejecutado correctamente (Marta → carlos con CA).
